In [ ]:
import os
import sys
import PIL
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import glob
import csv
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML
# Display plots inside the notebook:
%matplotlib inline 

In [ ]:
"""-------------------------------- MISCELLANEOUS FUNCTIONS -------------------------------------"""

# Plot a single color image
def plot_side_by_side(left_image, right_image, left_title='', right_title=''):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))    
    ax1.set_title(left_title, fontsize=30)
    ax1.imshow(left_image)    
    ax2.set_title(right_title, fontsize=30)
    ax2.imshow(right_image)

# Plot two grayscaled images side-by-side
def plot_side_by_side_gray(left_image, right_image, left_title='', right_title=''):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))    
    ax1.set_title(left_title, fontsize=30)
    ax1.imshow(left_image, cmap='gray')    
    ax2.set_title(right_title, fontsize=30)
    ax2.imshow(right_image, cmap='gray')    
    
# Read in images from folder:
def get_images_from_folder(folder_name='camera_cal'):
    cwd = os.getcwd()
    path = '{}/{}/{}'.format(cwd, folder_name, 'calibration*.jpg')
    image_paths = glob.glob(path)
    images = []
    for image_path in image_paths:
        image = mpimg.imread(image_path)
        images.append(image)
    return images


"""---------------------------------- CAMERA CALIBRATION ------------------------------------------"""

# Calibrate camera: 
# NOTES: board size = (# horizontal squares, # vertical squares)
def calibrate_camera(folder_in='camera_cal', folder_out='camera_cal_output', board_size=(7,10)):
    print('Calibrating', end='')
    nx = board_size[1]-1 # num vertical lines on the chessboard
    ny = board_size[0]-1 # num horizontal lines on the chessboard
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(m,n,0)
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
    
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('{}/calibration*.jpg'.format(folder_in))

    # Count the number of images for which corners were found
    num_found = 0
    
    # Step through the list and search for chessboard corners
    for idx, fname in enumerate(images):
        print('.', end='')
        img = cv2.imread(fname)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

        # If found, add object points and image points
        if ret == True:
            num_found += 1
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw the corners
            cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
            
            # Save images with found corners to file and display a sample image
            write_name = 'camera_cal_output/corners_found'+str(idx)+'.jpg'
            cv2.imwrite(write_name, img)
             
    # Get image size
    img_size = (img.shape[1], img.shape[0])

    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

    # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
    calibration_data = dict()
    calibration_data["mtx"] = mtx
    calibration_data["dist"] = dist
    calibration_data["board_size"] = board_size
    pickle.dump(calibration_data, open("calibration_data.p", "wb" ) )
    
    # Print out calibration results
    if num_found >= 1:
        print('\nCalibration successful!')
        print('\nChessboard corners were found on {} of {} images.\n'.format(num_found,len(images)))        
        plot_side_by_side(img, undistort(img, mtx, dist))
    else:
        print('\nCalibration failed because it could not find any chessboard corners.\n')
    
    
    
"""---------------------------------- IMAGE PIPELINE ------------------------------------------"""

# STEP 1: Undistort image
def undistort(img, mtx, dist, display_results=False):

    undistorted_image = cv2.undistort(img, mtx, dist, None, mtx)
    return undistorted_image


# STEP 2: Warp Image.  
def trapezoid_overlay(img, vertices, thickness=2):
    original_img = np.copy(img)
    trapezoid_img = cv2.polylines(original_img, [vertices.astype('int32')], True, (50,205,50), thickness)
    return trapezoid_img


def warp(img, src, dst):
    # Choose offset from image corners to plot detected corners
    # NOTE: This should be chosen to present the result at the proper aspect ratio

    # Get the image shape (width (pos-right), height (pos-down))
    img_size = (img_w, img_h)
   
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    #Minv = cv2.getPerspectiveTransform(dst, src)   
    
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size)   
    
    return warped


# STEP 3: Create a color and Sobel threshold image
def color_and_gradient_mask(img):
    
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    
    # Convert to HLS color space and separate the S channel
    hls_s_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:,:,2]
    
    # Threshold 'hls_s' color channel
    hls_s_thresh_min = 220 #100
    hls_s_thresh_max = 250 #255 
    hls_s_binary = np.zeros_like(hls_s_channel)
    hls_s_binary[(hls_s_channel >= hls_s_thresh_min) & (hls_s_channel <= hls_s_thresh_max)] = 1
    
    
    # Convert to LUV color space and separate the L channel
    luv_l_channel = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)[:,:,0]
    
    # Threshold 'luv_l' color channel
    luv_l_thresh_min = 225# 225
    luv_l_thresh_max = 250# 255
    luv_l_binary = np.zeros_like(luv_l_channel)
    luv_l_binary[(luv_l_channel >= luv_l_thresh_min) & (luv_l_channel <= luv_l_thresh_max)] = 1
    
    # Create a grayscale image for Sobel
    gray = np.copy(hls_s_channel)
    
    # Find Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in the x-direction
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = 20 
    thresh_max = 255 
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    
    # Stack each channel to view their individual contributions in green and blue respectively
    # Green = HLS S-Channel component
    # Blue  = LUV L-channel component
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(hls_s_binary), hls_s_binary, luv_l_binary)) * 255

    
    # Combine the two color binary images along with the Sobel-X binary image into one binary image
    combined_binary = np.zeros_like(hls_s_binary)
    combined_binary[(hls_s_binary == 1) | (luv_l_binary == 1) | (sxbinary == 1)] = 1

    return combined_binary, hls, hls_s_channel, color_binary 

# STEP 4: Search

# Plot a histogram: 
def plot_histogram(histogram, histogram_title='', figure_size=(10,5)):
    plt.figure(figsize=figure_size)
    plt.title(histogram_title, fontsize=30)
    plt.plot(histogram)
        
        
# Make a histogram:
def make_histogram(binary, padding=100, display_results=False):
     # Take a histogram of the bottom half of the image
    histogram = np.sum(binary[binary.shape[0]//2:,:], axis=0)
    # Chop off any histogram values near the edge of the image 
    histogram[:padding] = 0 
    histogram[:len(histogram)-padding-1:-1] = 0   
    
    if display_results==True:
        plot_histogram(histogram, 'Histogram')
    return histogram


# Perform a histogram-based window search
# NOTE: This method is only used for the first frame of the input video
def window_search(warped_binary, histogram, dfc_correction_factor=0.3, display_results=False):
    
    # Create a blank output image to draw on and visualize the result
    out_img = np.dstack((warped_binary, warped_binary, warped_binary))*255

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint]) # 
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(warped_binary.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = warped_binary.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = warped_binary.shape[0] - (window+1)*window_height
        win_y_high = warped_binary.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
        (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
        (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, warped_binary.shape[0]-1, warped_binary.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    
    # Get radius of curvature (average of left and right lines) [meters]
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    avg_curverad = (left_curverad + right_curverad)/2
    
    
    # Get offset (distance from center of vehicle to center of lane) [meters]
    # negative offset: left-of-center
    # positive offset: right-of-center
    left_line_pos = left_fitx[-1]
    right_line_pos = right_fitx[-1]
    lane_center_pos = ((right_line_pos - left_line_pos)/2) + left_line_pos
    vehicle_center_pos = warped_binary.shape[1]/2
    distance_from_center = ((vehicle_center_pos - lane_center_pos)*xm_per_pix) + dfc_correction_factor
    
    # Create black image with only lines on it
    line_fits_binary = np.zeros((warped_binary.shape[0], warped_binary.shape[1]), dtype='uint8')
    for i in range(len(ploty)):
        try:
            index = (int(ploty[i]), int(left_fitx[i]))
            line_fits_binary[index[0], index[1]] = 255
            index = (int(ploty[i]), int(right_fitx[i]))
            line_fits_binary[index[0], index[1]] = 255
        except:
            pass

    if display_results==True:
        
        # ONLY USED FOR WRITEUP VISUALIZATION
        # Visualization
        # Generate x and y values for plotting
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        plt.figure(figsize=(10,5))
        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.title('Window Search', fontsize=30)
        plt.xlim(0, 1280)
        plt.ylim(720, 0)

    return line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, out_img


# Perform a proximity search
# NOTE: This method will be used for every frame other than the first frame of the input video
def proximity_search(warped_binary, left_fit, right_fit, dfc_correction_factor=0.3, display_results=False):
    # Assume you now have a new warped binary image 
    # from the next frame of video (also called "warped_binary")
    # It's now much easier to find line pixels!
    # Create an image to draw on and an image to show the selection window
    
    out_img = np.dstack((warped_binary, warped_binary, warped_binary))*255
    nonzero = warped_binary.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]    
       
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, warped_binary.shape[0]-1, warped_binary.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Get radius of curvature (average of left and right lines)
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    avg_curverad = (left_curverad + right_curverad)/2

    # Get offset (distance from center of vehicle to center of lane) [meters]
    # negative offset: left-of-center
    # positive offset: right-of-center
    left_line_pos = left_fitx[-1]
    right_line_pos = right_fitx[-1]
    lane_center_pos = ((right_line_pos - left_line_pos)/2) + left_line_pos
    vehicle_center_pos = warped_binary.shape[1]/2
    distance_from_center = ((vehicle_center_pos - lane_center_pos)*xm_per_pix) + dfc_correction_factor
    
    # Create black image with only lines on it
    line_fits_binary = np.zeros((warped_binary.shape[0], warped_binary.shape[1]), dtype='uint8')
    for i in range(len(ploty)):
        try:
            index = (int(ploty[i]), int(left_fitx[i]))
            line_fits_binary[index[0], index[1]] = 255
            index = (int(ploty[i]), int(right_fitx[i]))
            line_fits_binary[index[0], index[1]] = 255
        except:
            pass   
    
   
    # ONLY FOR WRITEUP VISUALIZATION
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((warped_binary, warped_binary, warped_binary))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))



    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    if display_results == True:
        plt.figure(figsize=(10,5))
        plt.imshow(result)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.title('Proximity Search', fontsize=30)
        plt.xlim(0, 1280)
        plt.ylim(720, 0)

    return line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, result


# STEP 5: Create an image from the averaged polyline fits 
def avg_fits(line_fits_binary, ploty, avg_left_fitx, avg_right_fitx):
    avg_line_fits_binary = np.zeros((line_fits_binary.shape[0], line_fits_binary.shape[1]), dtype='uint8')
    for i in range(len(ploty)):
        try:
            index = (int(ploty[i]), int(avg_left_fitx[i]))
            avg_line_fits_binary[index[0], index[1]] = 255
            index = (int(ploty[i]), int(avg_right_fitx[i]))
            avg_line_fits_binary[index[0], index[1]] = 255
        except:
            pass
    return avg_line_fits_binary


# STEP 6:  Fill the warped trapezoid 
def fill_image(line_fits_binary, ploty, left_fitx, right_fitx):
    
    warped = np.copy(line_fits_binary)

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    filled_image = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(filled_image, np.int_([pts]), (0,255, 0)) # <-- color will be green

    return filled_image


# STEP 7: Unwarp the filled in image
def unwarp(img, src, dst):
  
    # Given src and dst points, calculate the perspective transform matrix
    #M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)   
    
    # Warp the image using OpenCV warpPerspective()
    unwarped = cv2.warpPerspective(img, Minv, img_size)   
    return unwarped


# STEP 8:  Reshape the unwarped image so that it has 3 color channels instead of 1
def reshape_unwarped(unwarped_image, display_results=False):
    reshaped_unwarped_image = np.zeros((unwarped_image.shape[0], unwarped_image.shape[1], 3), dtype='uint8')    
    # Fill shape using the green color channel
    #reshaped_unwarped[:,:,0] = unwarped[:,:]
    reshaped_unwarped_image[:,:,1] = unwarped_image[:,:]
    #reshaped_unwarped[:,:,2] = unwarped[:plt.imshow(reshaped_unwarped)   
    
    if display_results==True:
        plot_single_image(reshaped_unwarped_image, 'Reshaped Unwarped Image')
    return reshaped_unwarped_image


# STEP 9:  Combine the filled in trapezoid with the original image 
def weigh_image(reshaped_unwarped_image, original_image, α=0.8, β=1., λ=0., display_results=False):    
    weighted_image = cv2.addWeighted(original_image, α, reshaped_unwarped_image, β, λ)
    
    if display_results==True:
        img = np.copy(weighted_image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plot_single_image(img, 'Weighted Image')
    
    return weighted_image


# STEP 10:  Add text overlay to weighted image
# https://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html?highlight=puttext#void putText(Mat& img, const string& text, Point org, int fontFace, double fontScale, Scalar color, int thickness, int lineType, bool bottomLeftOrigin)
def text_overlay(weighted_image, avg_curverad, distance_from_center, display_results=False):
    img = np.copy(weighted_image)
    roc = str(int(avg_curverad))
    dfc = str(round(abs(distance_from_center), 3))

    if distance_from_center <= 0:
        direction = "LEFT OF CENTER"
    else:
        direction = "RIGHT OF CENTER"

    text1 = "RADIUS OF CURVATURE = {} METERS".format(roc)
    text2 = "VEHICLE IS {} METERS {}".format(dfc, direction)
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    final_image = cv2.putText(img, text1, (50, 80), font, 1.4, (255, 255, 255), 2, cv2.LINE_AA)
    final_image = cv2.putText(img, text2, (50, 130), font, 1.4, (255, 255, 255), 2, cv2.LINE_AA)

    if display_results==True:
        img = np.copy(final_image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
        plot_single_image(img, 'Final Image')
    
    return final_image



# Camera Calibration

In [ ]:
# Calibrate camera and store calibration data in .p file

# NOTE: This step can be skipped calibration has already been
#       performed for this camera.  The 'mtx' and 'dist' variables
#       will be read in from the 'calibration_data.p' file.

calibrate_camera()

In [ ]:
# Get camera calibration data from pickle file:
calibration_data = pickle.load( open("calibration_data.p", "rb" ) )
mtx, dist = calibration_data["mtx"], calibration_data["dist"]

# Image Pipeline

In [ ]:
# STEP 0: Read in a test image
original_image = cv2.imread('test_images/straight_lines2.jpg')
original_image = gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
image_height = original_image.shape[0]
image_width = original_image.shape[1]
num_channels = original_image.shape[2]

# Save some useful image dimensions
img_h = original_image.shape[0]
img_w = original_image.shape[1]
y_mid = img_h//2
x_mid = img_w//2

# Plot the image
plt.figure(figsize=(10,5))
plt.imshow(original_image, cmap=None)

In [ ]:
# STEP 1: Undistort image
undistorted_image = undistort(original_image, mtx, dist)
plot_side_by_side(original_image, undistorted_image, 'Original', 'Undistorted')

In [ ]:
# STEP 2: Apply a perspective transform to get a warped "bird's eye view" image

# Define trapezoid dimensions
trap_height = 260
trap_top_width = 170
trap_base_width = img_w-100
trap_bottom_cutoff = 40
trap_v_shift = 0   # positive --> shifts entire trapezoid UP
trap_h_shift = 0   # positive --> shifts entire trapezoid to the RIGHT

# Store 4 trapezoid vertices in 'src' (...starting with top left point going clockwise)
src = np.float32(
    [[x_mid + trap_h_shift - trap_top_width//2, img_h - trap_v_shift - trap_height],
     [x_mid + trap_h_shift + trap_top_width//2, img_h - trap_v_shift - trap_height],
     [x_mid + trap_h_shift + trap_base_width//2, img_h - trap_v_shift - trap_bottom_cutoff],
     [x_mid + trap_h_shift - trap_base_width//2, img_h - trap_v_shift - trap_bottom_cutoff]])

# The 4 points from 'src' will be mapped to these 4 points
# NOTE: 'offset' is padding added to the left and right sides of the warped image
top_offset = 200
base_offset = top_offset - 60
img_size = (img_w, img_h)

# The 4 trapezoid vertices will be remapped to these 'dst' points
dst = np.float32([[top_offset, 0],
                  [img_w-top_offset, 0], 
                  [img_w-base_offset, img_h], 
                  [base_offset, img_h]])

# Warp the image
warped = warp(undistorted_image, src,dst)
plot_side_by_side_gray(undistorted_image, warped ,'Undistorted' , 'Warped')


In [ ]:
# STEP 3: Apply color and Sobel thresholds to create a combined_binary image
combined_binary, hls, s_channel, color_binary = color_and_gradient_mask(warped)
plot_side_by_side_gray(color_binary, combined_binary, 'HLS-S(green) and LUV-L(blue)', 'HLS-S, LUV-L, and Sobel-X Binary')

In [ ]:
histogram = make_histogram(combined_binary, 200, True)

In [ ]:
line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, out_img = window_search(combined_binary, histogram, dfc_correction_factor=0.3, display_results=True)
#plt.imshow(line_fits_binary, cmap='gray')

In [ ]:
line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, out_img = proximity_search(combined_binary, left_fit, right_fit, dfc_correction_factor=0.3, display_results=True)
#plt.imshow(line_fits_binary, cmap='gray')

In [ ]:
filled_image = fill_image(line_fits_binary, ploty, left_fitx, right_fitx)
plt.imshow(filled_image, cmap='gray')

In [ ]:
unwarped_image = unwarp(filled_image, src, dst)
plt.imshow(unwarped_image, cmap='gray')

In [ ]:
weighted_image = weigh_image(unwarped_image, original_image, α=1., β=.5, λ=0.)
plt.imshow(weighted_image)

In [ ]:
final_image = text_overlay(weighted_image, avg_curverad, distance_from_center)
plt.imshow(final_image)

# Process Test Images

In [ ]:
# Read in images from a folder in the current working directory
def read_in_image(path_in, display_results=False):    
    original_image = cv2.imread(path_in)    
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    
    if display_results==True:
        plot_single_image(original_image, 'Original Image')
    return original_image

In [ ]:
# Run a test image throught the pipeline.  Save intermediate steps to .jpg file.
def process_test_image(folder_in, image_name, folder_out, display_results=True):
       
    path_in = "{}/{}".format(folder_in, image_name)
    subfolder_out = image_name.split('.')[0]
     
    original_image = read_in_image(path_in, display_results=False)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '0_original_image.jpg')    
    cv2.imwrite(path, original_image)    
    
    undistorted_image = undistort(original_image, mtx, dist)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '1_undistorted_image.jpg')    
    cv2.imwrite(path, undistorted_image)    
    
    trapezoid_image = trapezoid_overlay(undistorted_image, src)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '2_trapezoid_image.jpg')    
    cv2.imwrite(path, trapezoid_image)        
    
    combined_binary, hls, s_channel, color_binary = color_and_gradient_mask(undistorted_image)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '3_color_binary.jpg')    
    cv2.imwrite(path, color_binary)    
    
    warped = warp(combined_binary, src, dst)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '4_warped.jpg')    
    warped_save = (np.copy(warped))*255    
    cv2.imwrite(path, warped_save)    
    
    histogram = make_histogram(warped)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '5_histogram.jpg')    
    plt.figure(figsize=(10,5))
    plt.title('Histogram', fontsize=30)
    plt.plot(histogram)
    plt.savefig(path)
    plt.close()
        
    line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, window_search_image = window_search(warped, histogram, dfc_correction_factor=0.3)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '6_window_search_image.jpg')    
    cv2.imwrite(path, window_search_image)    
    
    line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, proximity_search_image = proximity_search(warped, left_fit, right_fit, dfc_correction_factor=0.3)   
    path = "{}/{}/{}".format(folder_out, subfolder_out, '7_proximity_search_image.jpg')    
    cv2.imwrite(path, proximity_search_image)    
    
    filled_image = fill_image(line_fits_binary, ploty, left_fitx, right_fitx)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '8_filled_image.jpg')    
    cv2.imwrite(path, filled_image)    
    
    unwarped_image = unwarp(filled_image, src, dst)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '9_unwarped_image.jpg')    
    cv2.imwrite(path, unwarped_image)
    
    weighted_image = weigh_image(unwarped_image, original_image, α=1.0, β=0.4, λ=0.0)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '10_weighted_image.jpg')    
    cv2.imwrite(path, weighted_image)
    
    final_test_image = text_overlay(weighted_image, avg_curverad, distance_from_center)
    path = "{}/{}/{}".format(folder_out, subfolder_out, '11_final_test_image.jpg')    
    cv2.imwrite(path, final_test_image)
    
    if display_results==True:
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        final_test_image = cv2.cvtColor(final_test_image, cv2.COLOR_BGR2RGB)
        
        plot_side_by_side(original_image, final_test_image, 'Original Image', 'Final Test Image')   
    
    return final_test_image


In [ ]:
# Run all test images through pipeline and save images to folder_out:
folder_in = 'test_images'
folder_out = 'output_images'

# Get a list of test image names:
image_names = os.listdir(folder_in)

# Create output subfolders for each test image:
for image_name in image_names:
    subfolder_out = "{}/{}".format(folder_out, image_name.split('.')[0])
    if not os.path.exists(subfolder_out):
        os.makedirs(subfolder_out)
        
    final_test_image = process_test_image(folder_in, image_name, folder_out, display_results=True)   
   


# Process Videos

In [ ]:
# Create a Frame class to store information pertaining to each frame in the video
class Frame():
    
    # Maximum and Minimum allowable polynomial curve coefs.
    # Polynomials are in the form: avg_fit = [a, b, c] (...where: a*x^2 + bx + c )
    max_a = 4.0e-4 # Determines maximum "curviness" of the quadratic polynomial
    min_a = 0 
    max_b = 0.3 # Determines maximum slope of the quadratic polynomial
    min_b = 0.1
    max_c = 300 # Determines maximum intercept of the quadratic polynomial
    min_c = 200
    # Count the number of times 
    a_fail_count = 0
    b_fail_count = 0
    c_fail_count = 0
    curvature_fail_count = 0
    
    
#     # Maximum allowable CHANGE in polynomial curve coefs from previous frame to current frame
#     # Polynomials are in the form: a*x^2 + bx + c --> avg_fit = [a, b, c]
    max_delta_a = 10000
    max_delta_b = 3.6e-1
    max_delta_c = 115
    delta_a_fail_count = 0
    delta_b_fail_count = 0
    delta_c_fail_count = 0

    num_old_fits_retained = 3 # 3
    old_left_fits = []
    old_right_fits = []
    ploty = np.array([False])
    
    old_left_fitx = []
    old_right_fitx = []
    
    num_failed_searches = 0
    num_failed_deltas = 0
    num_frames = 0
    
    @classmethod
    def add_left_fit_to_list(cls, left_fit):
        if (len(cls.old_left_fits) >= cls.num_old_fits_retained):
            cls.old_left_fits.append(left_fit)
            cls.old_left_fits.pop(0)
        else:
            cls.old_left_fits.append(left_fit)

    @classmethod
    def add_right_fit_to_list(cls, right_fit):
        if (len(cls.old_right_fits) >= cls.num_old_fits_retained):
            cls.old_right_fits.append(right_fit)
            cls.old_right_fits.pop(0)
        else:
            cls.old_right_fits.append(right_fit)
    
    @classmethod
    def add_left_fitx_to_list(cls, left_fitx):
        if (len(cls.old_left_fitx) >= cls.num_old_fits_retained):
            cls.old_left_fitx.append(left_fitx)
            cls.old_left_fitx.pop(0)
        else:
            cls.old_left_fitx.append(left_fitx)
        
    @classmethod
    def add_right_fitx_to_list(cls, right_fitx):
        if (len(cls.old_right_fitx) >= cls.num_old_fits_retained):
            cls.old_right_fitx.append(right_fitx)
            cls.old_right_fitx.pop(0)
        else:
            cls.old_right_fitx.append(right_fitx)
        
    @classmethod       
    def reset(cls):
        cls.num_frames = 0
        cls.old_left_fits = []
        cls.old_right_fits = []
        cls.num_failed_searches = 0
        cls.num_failed_deltas = 0
        cls.a_fail_count = 0
        cls.b_fail_count = 0
        cls.c_fail_count = 0
        cls.delta_a_fail_count = 0
        cls.delta_b_fail_count = 0
        cls.delta_c_fail_count = 0
        cls.curvature_fail_count = 0

        
        
        
    def __init__(self):
            
        self.line_fits_binary = np.array([False])
        self.combined_binary = np.array([False])
        
        # Lines are detected or not detected in current frame
        self.left_detected = False
        self.right_detected = False
        
        # If lines were detected for current frame, these are the fitted polynomial coefs for left, right, and avg
        self.left_fitx = np.array([False])
        self.right_fitx = np.array([False])
        self.left_fit = np.array([0,0,0], dtype='float64')
        self.right_fit = np.array([0,0,0], dtype='float64')
        self.avg_curverad = np.array([0,0,0], dtype='float64')
        self.distance_from_center = np.array([0,0,0], dtype='float64')
        
        self.left_fit_diffs = np.array([0, 0, 0], dtype='float64')
        self.right_fit_diffs = np.array([0, 0, 0], dtype='float64')
               
        self.avg_left_fitx = []
        self.avg_right_fitx = []
        
        self.left_curvature_allowable = True
        self.right_curvature_allowable = True  
        self.left_does_not_cross_middle = True
        self.right_does_not_cross_middle = True
        
        self.left_middle_cross_fail_count = 0
        self.right_middle_cross_fail_count = 0
        
        self.curve_direction_same = True
        self.curve_direction_fail = 0
        
    

In [ ]:

### Process video functions
def process_image(img):    

    global first_frame
    
    copied_image = np.copy(img)
    undistorted_image = undistort(copied_image, mtx, dist)

    warped = warp(undistorted_image, src, dst)  

    combined_binary, hls, s_channel, color_binary = color_and_gradient_mask(warped)

    histogram = make_histogram(combined_binary)

    # Use window_search the first time through.  Use proximity search thereafter
    if first_frame == True:
        line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, search_image = window_search(combined_binary, histogram, dfc_correction_factor=0.3)
        first_frame = False
        
        previous_frame.left_fitx = left_fitx
        previous_frame.right_fitx = right_fitx
        previous_frame.line_fits_binary = line_fits_binary
        previous_frame.combined_binary = combined_binary
        previous_frame.left_fit = left_fit
        previous_frame.right_fit = right_fit
        previous_frame.avg_curverad = avg_curverad
        previous_frame.distance_from_center = distance_from_center
        
        Frame.ploty = ploty
        current_frame.left_fitx = left_fitx
        current_frame.right_fitx = right_fitx
        current_frame.line_fits_binary = line_fits_binary
        current_frame.combined_binary = combined_binary
        current_frame.left_fit = left_fit
        current_frame.right_fit = right_fit
        current_frame.avg_curverad = avg_curverad
        current_frame.distance_from_center = distance_from_center
        
        # Update the fits lists:
        Frame.add_left_fit_to_list(current_frame.left_fit)
        Frame.add_left_fitx_to_list(current_frame.left_fitx)
        Frame.add_right_fit_to_list(current_frame.right_fit)
        Frame.add_right_fitx_to_list(current_frame.right_fitx)
        
        # Calculate the average values from lists
        current_frame.avg_left_fitx = np.mean(Frame.old_left_fitx, axis=0)
        current_frame.avg_right_fitx = np.mean(Frame.old_right_fitx, axis=0)

    else:
        
        # Update previous frame.
        previous_frame.left_fitx = current_frame.left_fitx
        previous_frame.right_fitx = current_frame.right_fitx
        previous_frame.line_fits_binary = current_frame.line_fits_binary
        previous_frame.combined_binary = current_frame.combined_binary
        previous_frame.left_fit = current_frame.left_fit
        previous_frame.right_fit = current_frame.right_fit
        previous_frame.avg_curverad = current_frame.avg_curverad
        previous_frame.distance_from_center = current_frame.distance_from_center
        previous_frame.left_fit_diffs = current_frame.left_fit_diffs
        previous_frame.right_fit_diffs = current_frame.right_fit_diffs
        
        line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, search_image = proximity_search(previous_frame.combined_binary, previous_frame.left_fit, previous_frame.right_fit, dfc_correction_factor=0.3)   

        current_frame.left_fit_diffs = abs(previous_frame.left_fit - left_fit)
        current_frame.right_fit_diffs = abs(previous_frame.right_fit - right_fit)
        

        # Check that LEFT a coef. (curvature) is within a reasonable range:
        a = abs(left_fit[0])
        if (a < Frame.max_a):
            current_frame.left_curvature_allowable = True
        else:
            current_frame.left_curvature_allowable = False
            current_frame.curvature_fail_count += 1
        
        # Check that RIGHT a coef. (curvature) is within a reasonable range:
        a = abs(right_fit[0])
        if (a < Frame.max_a):
            current_frame.right_curvature_allowable = True
        else:
            current_frame.right_curvature_allowable = False
            current_frame.curvature_fail_count += 1
            
        # Check that LEFT and RIGHT a coef. have the same sign
        if ((left_fit[0] < 0 and right_fit[0] < 0) or (left_fit[0] > 0 and right_fit[0] > 0)):
            current_frame.curve_direction_same = True
        else:
            current_frame.curve_direction_same = False
            current_frame.curve_direction_fail += 1

#         # Check that LEFT line coefs. are not siginificantly different from previous frames coefs.
#         deltas = abs(left_fit - previous_frame.left_fit)
#         delta_a = deltas[0]
#         delta_b = deltas[1]
#         delta_c = deltas[2]
#         if (delta_a < Frame.max_delta_a):
#             if (delta_b < Frame.max_delta_b):
#                 if (delta_c < Frame.max_delta_c):
#                     current_frame.left_fit = left_fit
#                     current_frame.left_fitx = left_fitx
#                 else:
#                     Frame.delta_c_fail_count += 1
#             else:
#                 Frame.delta_b_fail_count += 1
#         else:
#             Frame.delta_a_fail_count += 1
                    
#          # Check that RIGHT line coefs. are not siginificantly different from previous frames coefs.
#         deltas = abs(right_fit - previous_frame.right_fit)
#         delta_a = deltas[0]
#         delta_b = deltas[1]
#         delta_c = deltas[2]
#         if (delta_a < Frame.max_delta_a):
#             if (delta_b < Frame.max_delta_b):
#                 if (delta_c < Frame.max_delta_c):
#                     current_frame.right_fit = right_fit
#                     current_frame.right_fitx = right_fitx
#                 else:
#                     Frame.delta_c_fail_count += 1
#             else:
#                 Frame.delta_b_fail_count += 1
#         else:
#             Frame.delta_a_fail_count += 1
            
    
        right_buffer = 0
        left_buffer = 0
        # Check that left_fitx points are not too crazy:
        if (max(left_fitx) < img_w//2 - left_buffer and min(left_fitx) > 0):
            current_frame.left_does_not_cross_middle = True
        else:
            current_frame.left_does_not_cross_middle = False
            current_frame.left_middle_cross_fail_count += 1
            
        # Check that right_fitx points are not too crazy:
        if (max(right_fitx) < img_w and min(right_fitx) > img_w//2 + right_buffer):
            current_frame.right_does_not_cross_middle = True
        else:
            current_frame.right_does_not_cross_middle = False
            current_frame.right_middle_cross_fail_count += 1
            
            
        # If they pass, update current_frame
        # If they fail, do nothing (current_frame values will stay equal to previous_frame values)    
        if (current_frame.left_curvature_allowable and current_frame.right_curvature_allowable and current_frame.left_does_not_cross_middle and current_frame.right_does_not_cross_middle and current_frame.curve_direction_same):
            current_frame.left_fit = left_fit
            current_frame.left_fitx = left_fitx
            current_frame.right_fit = right_fit
            current_frame.right_fitx = right_fitx
 
        # Update current frame values with the values returned by the search function
        current_frame.line_fits_binary = line_fits_binary
        current_frame.combined_binary = combined_binary
        current_frame.avg_curverad = avg_curverad
        current_frame.distance_from_center = distance_from_center
        
        # Update the fit lists:
        Frame.add_left_fit_to_list(current_frame.left_fit)
        Frame.add_left_fitx_to_list(current_frame.left_fitx)
        Frame.add_right_fit_to_list(current_frame.right_fit)
        Frame.add_right_fitx_to_list(current_frame.right_fitx)
        
        # Calculate the average values from lists
        current_frame.avg_left_fitx = np.mean(Frame.old_left_fitx, axis=0)
        current_frame.avg_right_fitx = np.mean(Frame.old_right_fitx, axis=0)

        
    # Create an avg_line_fits_binary to send to fill_image:
    avg_line_fits_binary = avg_fits(line_fits_binary, Frame.ploty, current_frame.avg_left_fitx, current_frame.avg_right_fitx)
    
    filled_image = fill_image(avg_line_fits_binary, Frame.ploty, current_frame.avg_left_fitx, current_frame.avg_right_fitx)

    unwarped_image = unwarp(filled_image, src, dst)

    weighted_image = weigh_image(unwarped_image, img, α=1.0, β=0.4, λ=0.0)

    final_image = text_overlay(weighted_image, current_frame.avg_curverad, current_frame.distance_from_center)
        
    Frame.num_frames += 1
    return final_image


In [ ]:
# # ### Process video functions
# # def process_image_color_binary(img):    

### Process video functions and output color_binary video
def process_image_color_binary(img):    

    global first_frame
    
    copied_image = np.copy(img)
    undistorted_image = undistort(copied_image, mtx, dist)

    warped = warp(undistorted_image, src, dst)  

    combined_binary, hls, s_channel, color_binary = color_and_gradient_mask(warped)

    histogram = make_histogram(combined_binary)

    # Use window_search the first time through.  Use proximity search thereafter
    if first_frame == True:
        line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, search_image = window_search(combined_binary, histogram, dfc_correction_factor=0.3)
        first_frame = False
        
        previous_frame.left_fitx = left_fitx
        previous_frame.right_fitx = right_fitx
        previous_frame.line_fits_binary = line_fits_binary
        previous_frame.combined_binary = combined_binary
        previous_frame.left_fit = left_fit
        previous_frame.right_fit = right_fit
        previous_frame.avg_curverad = avg_curverad
        previous_frame.distance_from_center = distance_from_center
        
        Frame.ploty = ploty
        current_frame.left_fitx = left_fitx
        current_frame.right_fitx = right_fitx
        current_frame.line_fits_binary = line_fits_binary
        current_frame.combined_binary = combined_binary
        current_frame.left_fit = left_fit
        current_frame.right_fit = right_fit
        current_frame.avg_curverad = avg_curverad
        current_frame.distance_from_center = distance_from_center
        
        # Update the fits lists:
        Frame.add_left_fit_to_list(current_frame.left_fit)
        Frame.add_left_fitx_to_list(current_frame.left_fitx)
        Frame.add_right_fit_to_list(current_frame.right_fit)
        Frame.add_right_fitx_to_list(current_frame.right_fitx)
        
        # Calculate the average values from lists
        current_frame.avg_left_fitx = np.mean(Frame.old_left_fitx, axis=0)
        current_frame.avg_right_fitx = np.mean(Frame.old_right_fitx, axis=0)

    else:
        
        # Update previous frame.
        previous_frame.left_fitx = current_frame.left_fitx
        previous_frame.right_fitx = current_frame.right_fitx
        previous_frame.line_fits_binary = current_frame.line_fits_binary
        previous_frame.combined_binary = current_frame.combined_binary
        previous_frame.left_fit = current_frame.left_fit
        previous_frame.right_fit = current_frame.right_fit
        previous_frame.avg_curverad = current_frame.avg_curverad
        previous_frame.distance_from_center = current_frame.distance_from_center
        previous_frame.left_fit_diffs = current_frame.left_fit_diffs
        previous_frame.right_fit_diffs = current_frame.right_fit_diffs
        
        line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, search_image = proximity_search(previous_frame.combined_binary, previous_frame.left_fit, previous_frame.right_fit, dfc_correction_factor=0.3)   

        current_frame.left_fit_diffs = abs(previous_frame.left_fit - left_fit)
        current_frame.right_fit_diffs = abs(previous_frame.right_fit - right_fit)
        
#       # Checks...
        # Check that LEFT a coef. (curvature) is within a reasonable range:
        a = abs(left_fit[0])
        if (a < Frame.max_a):
            current_frame.left_curvature_allowable = True
        else:
            current_frame.left_curvature_allowable = False
            current_frame.curvature_fail_count += 1
        
        # Check that RIGHT a coef. (curvature) is within a reasonable range:
        a = abs(right_fit[0])
        if (a < Frame.max_a):
            current_frame.right_curvature_allowable = True
        else:
            current_frame.right_curvature_allowable = False
            current_frame.curvature_fail_count += 1
            
        # Check that LEFT and RIGHT a coef. have the same sign
        if ((left_fit[0] < 0 and right_fit[0] < 0) or (left_fit[0] > 0 and right_fit[0] > 0)):
            current_frame.curve_direction_same = True
        else:
            current_frame.curve_direction_same = False
            current_frame.curve_direction_fail += 1

            
        right_buffer = 0
        left_buffer = 0
        # Check that left_fitx points are not too crazy:
        if (max(left_fitx) < img_w//2 - left_buffer and min(left_fitx) > 0):
            current_frame.left_does_not_cross_middle = True
        else:
            current_frame.left_does_not_cross_middle = False
            current_frame.left_middle_cross_fail_count += 1
            
        # Check that right_fitx points are not too crazy:
        if (max(right_fitx) < img_w and min(right_fitx) > img_w//2 + right_buffer):
            current_frame.right_does_not_cross_middle = True
        else:
            current_frame.right_does_not_cross_middle = False
            current_frame.right_middle_cross_fail_count += 1
            
            
        # If they pass, update current_frame
        # If they fail, do nothing (current_frame values will stay equal to previous_frame values)    
        if (current_frame.left_curvature_allowable and current_frame.right_curvature_allowable and current_frame.left_does_not_cross_middle and current_frame.right_does_not_cross_middle and current_frame.curve_direction_same):
            current_frame.left_fit = left_fit
            current_frame.left_fitx = left_fitx
            current_frame.right_fit = right_fit
            current_frame.right_fitx = right_fitx
   

        current_frame.line_fits_binary = line_fits_binary
        current_frame.combined_binary = combined_binary
        current_frame.avg_curverad = avg_curverad
        current_frame.distance_from_center = distance_from_center
        
        # Update the fits lists:
        Frame.add_left_fit_to_list(current_frame.left_fit)
        Frame.add_left_fitx_to_list(current_frame.left_fitx)
        Frame.add_right_fit_to_list(current_frame.right_fit)
        Frame.add_right_fitx_to_list(current_frame.right_fitx)
        
        # Calculate the average values from lists
        current_frame.avg_left_fitx = np.mean(Frame.old_left_fitx, axis=0)
        current_frame.avg_right_fitx = np.mean(Frame.old_right_fitx, axis=0)

        
#     # Create an avg_line_fits_binary to send to fill_image:
#     avg_line_fits_binary = avg_fits(line_fits_binary, Frame.ploty, current_frame.avg_left_fitx, current_frame.avg_right_fitx)
    
#     filled_image = fill_image(avg_line_fits_binary, Frame.ploty, current_frame.avg_left_fitx, current_frame.avg_right_fitx)

#     unwarped_image = unwarp(filled_image, src, dst)

#     weighted_image = weigh_image(unwarped_image, img, α=1.0, β=0.4, λ=0.0)

#     final_image = text_overlay(weighted_image, current_frame.avg_curverad, current_frame.distance_from_center)
        
    Frame.num_frames += 1
    return color_binary



In [ ]:
### Process video functions
def process_image_search(img):    

    global first_frame
    
    copied_image = np.copy(img)
    undistorted_image = undistort(copied_image, mtx, dist)

    warped = warp(undistorted_image, src, dst)  

    combined_binary, hls, s_channel, color_binary = color_and_gradient_mask(warped)

    histogram = make_histogram(combined_binary)

    # Use window_search the first time through.  Use proximity search thereafter
    if first_frame == True:
        line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, search_image = window_search(combined_binary, histogram, dfc_correction_factor=0.3)
        first_frame = False
        
        previous_frame.left_fitx = left_fitx
        previous_frame.right_fitx = right_fitx
        previous_frame.line_fits_binary = line_fits_binary
        previous_frame.combined_binary = combined_binary
        previous_frame.left_fit = left_fit
        previous_frame.right_fit = right_fit
        previous_frame.avg_curverad = avg_curverad
        previous_frame.distance_from_center = distance_from_center
        
        Frame.ploty = ploty
        current_frame.left_fitx = left_fitx
        current_frame.right_fitx = right_fitx
        current_frame.line_fits_binary = line_fits_binary
        current_frame.combined_binary = combined_binary
        current_frame.left_fit = left_fit
        current_frame.right_fit = right_fit
        current_frame.avg_curverad = avg_curverad
        current_frame.distance_from_center = distance_from_center
        
        # Update the fits lists:
        Frame.add_left_fit_to_list(current_frame.left_fit)
        Frame.add_left_fitx_to_list(current_frame.left_fitx)
        Frame.add_right_fit_to_list(current_frame.right_fit)
        Frame.add_right_fitx_to_list(current_frame.right_fitx)
        
        # Calculate the average values from lists
        current_frame.avg_left_fitx = np.mean(Frame.old_left_fitx, axis=0)
        current_frame.avg_right_fitx = np.mean(Frame.old_right_fitx, axis=0)

    else:
        
        # Update previous frame.
        previous_frame.left_fitx = current_frame.left_fitx
        previous_frame.right_fitx = current_frame.right_fitx
        previous_frame.line_fits_binary = current_frame.line_fits_binary
        previous_frame.combined_binary = current_frame.combined_binary
        previous_frame.left_fit = current_frame.left_fit
        previous_frame.right_fit = current_frame.right_fit
        previous_frame.avg_curverad = current_frame.avg_curverad
        previous_frame.distance_from_center = current_frame.distance_from_center
        previous_frame.left_fit_diffs = current_frame.left_fit_diffs
        previous_frame.right_fit_diffs = current_frame.right_fit_diffs
        
        line_fits_binary, left_fit, right_fit, ploty, left_fitx, right_fitx, avg_curverad, distance_from_center, search_image = proximity_search(previous_frame.combined_binary, previous_frame.left_fit, previous_frame.right_fit, dfc_correction_factor=0.3)   

        current_frame.left_fit_diffs = abs(previous_frame.left_fit - left_fit)
        current_frame.right_fit_diffs = abs(previous_frame.right_fit - right_fit)
        
#       # Checks...
        # Check that LEFT a coef. (curvature) is within a reasonable range:
        a = abs(left_fit[0])
        if (a < Frame.max_a):
            current_frame.left_curvature_allowable = True
        else:
            current_frame.left_curvature_allowable = False
            current_frame.curvature_fail_count += 1
        
        # Check that RIGHT a coef. (curvature) is within a reasonable range:
        a = abs(right_fit[0])
        if (a < Frame.max_a):
            current_frame.right_curvature_allowable = True
        else:
            current_frame.right_curvature_allowable = False
            current_frame.curvature_fail_count += 1
            
        # Check that LEFT and RIGHT a coef. have the same sign
        if ((left_fit[0] < 0 and right_fit[0] < 0) or (left_fit[0] > 0 and right_fit[0] > 0)):
            current_frame.curve_direction_same = True
        else:
            current_frame.curve_direction_same = False
            current_frame.curve_direction_fail += 1

            
        right_buffer = 0
        left_buffer = 0
        # Check that left_fitx points are not too crazy:
        if (max(left_fitx) < img_w//2 - left_buffer and min(left_fitx) > 0):
            current_frame.left_does_not_cross_middle = True
        else:
            current_frame.left_does_not_cross_middle = False
            current_frame.left_middle_cross_fail_count += 1
            
        # Check that right_fitx points are not too crazy:
        if (max(right_fitx) < img_w and min(right_fitx) > img_w//2 + right_buffer):
            current_frame.right_does_not_cross_middle = True
        else:
            current_frame.right_does_not_cross_middle = False
            current_frame.right_middle_cross_fail_count += 1
            
            
        # If they pass, update current_frame
        # If they fail, do nothing (current_frame values will stay equal to previous_frame values)    
        if (current_frame.left_curvature_allowable and current_frame.right_curvature_allowable and current_frame.left_does_not_cross_middle and current_frame.right_does_not_cross_middle and current_frame.curve_direction_same):
            current_frame.left_fit = left_fit
            current_frame.left_fitx = left_fitx
            current_frame.right_fit = right_fit
            current_frame.right_fitx = right_fitx
   

        current_frame.line_fits_binary = line_fits_binary
        current_frame.combined_binary = combined_binary
        current_frame.avg_curverad = avg_curverad
        current_frame.distance_from_center = distance_from_center
        
        # Update the fits lists:
        Frame.add_left_fit_to_list(current_frame.left_fit)
        Frame.add_left_fitx_to_list(current_frame.left_fitx)
        Frame.add_right_fit_to_list(current_frame.right_fit)
        Frame.add_right_fitx_to_list(current_frame.right_fitx)
        
        # Calculate the average values from lists
        current_frame.avg_left_fitx = np.mean(Frame.old_left_fitx, axis=0)
        current_frame.avg_right_fitx = np.mean(Frame.old_right_fitx, axis=0)

        
#     # Create an avg_line_fits_binary to send to fill_image:
#     avg_line_fits_binary = avg_fits(line_fits_binary, Frame.ploty, current_frame.avg_left_fitx, current_frame.avg_right_fitx)
    
#     filled_image = fill_image(avg_line_fits_binary, Frame.ploty, current_frame.avg_left_fitx, current_frame.avg_right_fitx)

#     unwarped_image = unwarp(filled_image, src, dst)

#     weighted_image = weigh_image(unwarped_image, img, α=1.0, β=0.4, λ=0.0)

#     final_image = text_overlay(weighted_image, current_frame.avg_curverad, current_frame.distance_from_center)
        
    Frame.num_frames += 1

    return search_image

In [ ]:
# Produce an output video (final_image):
def make_video(video_in_name, video_out_name, function_name, start_time=None, end_time=None):
    output = video_out_name
    clip1 = VideoFileClip(video_in_name)
    
    if start_time is not None:
        clip_out = clip1.fl_image(function_name).subclip(start_time, end_time) #NOTE: this function expects color images!!
    else:
        clip_out = clip1.fl_image(function_name) #NOTE: this function expects color images!!
    
    from IPython import get_ipython
    get_ipython().magic('time clip_out.write_videofile(output, audio=False)')
    
    # Show video output
    HTML("""
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(output))
    return output

In [ ]:
first_frame = True
Frame.reset()
start_time = 0
end_time = 50
previous_frame = Frame()
current_frame = Frame()

In [ ]:
first_frame = True
final_video = make_video('project_video.mp4','project_video_output.mp4', process_image, start_time, end_time)

print(previous_frame.left_fit)
print(current_frame.left_fit)
print(current_frame.left_fit_diffs)
print(current_frame.right_fit_diffs)
print(Frame.num_frames)
print(Frame.num_failed_deltas)

In [ ]:
first_frame = True
Frame.reset()
start_time = 0
end_time = 50
previous_frame = Frame()
current_frame = Frame()

In [ ]:
# Process challenge_video.mp4
first_frame = True
final_video = make_video('challenge_video.mp4','challenge_video_output.mp4', process_image, start_time, end_time)


In [ ]:
first_frame = True
Frame.reset()
start_time = 41
end_time = 43
previous_frame = Frame()
current_frame = Frame()

In [ ]:
# Green = HLS S-Channel component
# Blue  = LUV L-channel component
first_frame = True
color_binary_video = make_video('project_video.mp4','project_video_output_color_binary.mp4', process_image_color_binary, start_time, end_time)

print(previous_frame.left_fit)
print(current_frame.left_fit)
print(current_frame.left_fit_diffs)
print(current_frame.right_fit_diffs)
print(Frame.num_frames)
print(Frame.num_failed_deltas)

In [ ]:
first_frame = True
Frame.reset()
start_time = 41
end_time = 43
previous_frame = Frame()
current_frame = Frame()

In [ ]:
first_frame = True
search_video = make_video('project_video.mp4','project_video_output_search.mp4', process_image_search, start_time, end_time)

In [ ]:
######################################################################################################################################
######################################################################################################################################
######################################################################################################################################